# Docling + LangChain PDF Parsing Test
Dieses Notebook lädt alle PDFs aus einem Ordner mit DoclingLoader und analysiert die Verarbeitungsgeschwindigkeit.

## Installation

In [ ]:
!uv add langchain-docling

## Setup und Imports

In [ ]:
import os
import time

from langchain_docling import DoclingLoader

folder_path = "../src/docling/test_files/"
assert os.path.isdir(folder_path), f"Ordner nicht gefunden: {folder_path}"
pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]
assert pdf_files, f"Keine PDF-Dateien in {folder_path} gefunden."
print(f"{len(pdf_files)} PDF-Dateien gefunden.")

## Dokumente laden und Zeit messen

In [ ]:
all_documents = []
start_time = time.time()

for pdf_path in pdf_files:
    loader = DoclingLoader(pdf_path)
    docs = loader.load()
    all_documents.extend(docs)

duration = time.time() - start_time

print(f"Gesamt geladene Seiten: {len(all_documents)}")
print(f"⏱ Gesamtdauer: {duration:.2f} Sekunden")

## Performance-Schätzung für 1000 Seiten

In [ ]:
seiten = len(all_documents)
pro_seite = duration / seiten if seiten > 0 else 0
hochrechnung = pro_seite * 1000

print(f"Ø Zeit pro Seite: {pro_seite:.2f} s")
print(f"Geschätzte Zeit für 1000 Seiten: {hochrechnung / 60:.2f} Minuten")

## Beispieloutput anzeigen

In [ ]:
if all_documents:
    print("=== Beispiel: Erste Seite ===")
    print("Metadaten:", all_documents[0].metadata)
    print("Textauszug:", all_documents[0].page_content[:500])
else:
    print("⚠️ Keine Dokumente geladen.")

## Durch gesamten Ordner iterieren und Output in .md files speichern

In [ ]:
output_folder = "../src/docling/output_md_docling/"
os.makedirs(output_folder, exist_ok=True)

for idx, doc in enumerate(all_documents, start=1):
    base_name = os.path.basename(doc.metadata.get("source", f"doc_{idx}")).replace(".pdf", "")
    output_file = os.path.join(output_folder, f"{base_name}_page{idx}.md")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"# Metadaten\n{doc.metadata}\n\n")
        f.write(f"# Inhalt\n{doc.page_content}")

print(f"✅ {len(all_documents)} Seiten in Markdown-Dateien gespeichert unter: {output_folder}")

In [ ]:
# %%
from collections import defaultdict

output_folder = "../src/docling/output_md_docling/"
os.makedirs(output_folder, exist_ok=True)

# Dokumente nach Quelle gruppieren
grouped_docs = defaultdict(list)
for doc in all_documents:
    source = os.path.basename(doc.metadata.get("source", "unknown.pdf"))
    grouped_docs[source].append(doc)

# Pro PDF eine MD-Datei schreiben
for source, docs in grouped_docs.items():
    base_name = source.replace(".pdf", "")
    output_file = os.path.join(output_folder, f"{base_name}.md")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"# {base_name}\n\n")
        for idx, doc in enumerate(docs, start=1):
            f.write(doc.page_content)
    print(f"📄 Gespeichert: {output_file}")

print(f"✅ Alle PDFs als Markdown-Dateien exportiert nach: {output_folder}")